In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from datasource import getYahooFinanceData, getJugaadData, getFullData
from datemodule import getDateRange
from indicators import SupportResistanceIndicator

import plotly.graph_objects as go
import pandas as pd

In [3]:
ticker = 'WIPRO'
start, end = getDateRange('1y')
data = getFullData(ticker, start, end)

In [4]:
sri = SupportResistanceIndicator(data, 11, 5)
sri.calculate()

In [8]:
sri.showIndicator(250)

In [18]:
sri.proximity

0.8005578053146685